In [1]:
import cv2
import open3d as o3d
import numpy as np
import colorsys
import pyransac3d as pyrsc
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
show_pcl = True

In [3]:
def visualizer(cloud):

    if show_pcl:

        if type(cloud) == o3d.cpu.pybind.geometry.TriangleMesh:
            vertices = np.asarray(cloud.vertices)
            center_of_mass = np.mean(vertices, axis=0)
        else:
            # point cloud
            center_of_mass = np.mean(np.asarray(cloud.points), axis=0)
        
        # print("com", center_of_mass)
        o3d.visualization.draw_geometries([cloud],
                                    zoom=0.7,
                                    front=[1, 0, 0],
                                    lookat=center_of_mass,
                                    up=[0, 0, 1])
    
    else:
        pass


In [4]:
pcd = o3d.io.read_point_cloud("../3dmodels\Strawberry_Flowers_nice.ply")

# Cube 1 -> 8.13
# Cube 2 -> 11.86621893003967
# Cube 3 -> 7.35, 11.56398169309081

visualizer(pcd)

In [7]:
o3d.visualization.draw_geometries([pcd])


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [29]:
# # Visualize cloud and edit
# vis = o3d.visualization.VisualizerWithEditing()
# vis.create_window()
# vis.add_geometry(pcd)
# vis.run()
# vis.destroy_window()
# print(vis.get_picked_points())

In [30]:
bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=(-1, -1, -2),
                                                    max_bound=(0.5, 0, 1))
axis_bounded_cloud = pcd.crop(bounding_box)

visualizer(axis_bounded_cloud)
pcd = axis_bounded_cloud

In [31]:
o3d.visualization.draw_geometries([pcd])

In [7]:
pcd.scale(5, center=pcd.get_center())
visualizer(pcd)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [12]:
threshold = 0.75 # 0 -> 1

x, y, z = np.asarray(pcd.points)[:, :3].T

std = np.std((x, y, z), axis=1) * threshold

hist_x, bins_x = np.histogram(x, bins=50)
hist_y, bins_y = np.histogram(y, bins=50)

max_freq_bin_x = bins_x[np.argmax(hist_x)]
max_freq_bin_y = bins_y[np.argmax(hist_y)]

min_bound = (max_freq_bin_x, max_freq_bin_y, -2) - std
max_bound = (max_freq_bin_x, max_freq_bin_y, 2) + std

bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound,
                                                    max_bound=max_bound)
axis_bounded_cloud = pcd.crop(bounding_box)

visualizer(axis_bounded_cloud)

In [34]:
o3d.visualization.draw_geometries([axis_bounded_cloud])

In [35]:
def get_min_iter():
    s = 6  #no. of samples used every time to create the shape
    p = 0.99 #99%
    e = 0.5 # 50%
    k = np.log(1-p)/(np.log(1 - np.power((1-e), s)))
    print(k)
    return k
    # k => number of iterations

# get_min_iter()

In [36]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Generate random data for the histogram
# fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 4))

# axes[0].hist(x, bins=50, color='Yellow', edgecolor='black')
# axes[0].set_title('Histogram 1')
 
# axes[1].hist(y, bins=50, color='Pink', edgecolor='black')
# axes[1].set_title('Histogram 2')

# axes[2].hist(z, bins=50, color='Green', edgecolor='black')
# axes[2].set_title('Histogram 3')

# # Adding labels and title
# for ax in axes:
#     ax.set_xlabel('Values')
#     ax.set_ylabel('Frequency')
#     ax.xaxis.set_major_locator(plt.MaxNLocator(20))  # Adjust the number of ticks here
#     plt.setp(ax.get_xticklabels(), rotation=45)

# plt.tight_layout()
# plt.show()


In [37]:
# import matplotlib.pyplot as plt
# import numpy as np

# plt.hist2d(x, y, bins=30)

# plt.xlabel('X values')
# plt.ylabel('Y values')
# plt.title('2D Histogram (Hexbin Plot)')
# plt.colorbar()
# plt.show()


In [38]:
# point = pyrsc.Point()  #Using RANSAC
# # s = 1

# center, best_inliers_point = point.fit(np.asarray(axis_bounded_cloud.points), thresh=0.7, maxIteration=10)

# inlier_cloud_point = axis_bounded_cloud.select_by_index(best_inliers_point)
# outlier_cloud_point = axis_bounded_cloud.select_by_index(best_inliers_point, invert=True)

# # outlier_cloud_point.paint_uniform_color([1, 0, 0])
# # inlier_cloud_point.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([inlier_cloud_point],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [39]:
# sphere = pyrsc.Sphere()
# # s = 4
# center, radius, best_inliers_sphere = sphere.fit(np.asarray(axis_bounded_cloud.points), thresh=0.3, maxIteration=75)

# inlier_cloud_sphere = axis_bounded_cloud.select_by_index(best_inliers_sphere)
# outlier_cloud_sphere = axis_bounded_cloud.select_by_index(best_inliers_sphere, invert=True)

# outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
# inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_sphere, inlier_cloud_sphere],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [40]:
# # plane??

# sphere = pyrsc.Plane()
# # s = 3
# equation, best_inliers_plane = sphere.fit(np.asarray(inlier_cloud_point.points), thresh=0.05, maxIteration=40)

# inlier_cloud_plane = inlier_cloud_point.select_by_index(best_inliers_plane)
# outlier_cloud_plane = inlier_cloud_point.select_by_index(best_inliers_plane, invert=True)

# # outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# # inlier_cloud_plane.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_plane],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [13]:
pcd = axis_bounded_cloud

In [14]:
def convert_rgb_2_hsv(all_rgb_colors):
    all_hsv_colors = []

    for i in range(len(all_rgb_colors)):
        temp_color = all_rgb_colors[i]
        temp = colorsys.rgb_to_hsv(temp_color[0], temp_color[1], temp_color[2])
        all_hsv_colors.append(temp)

    all_hsv_colors = np.asarray(all_hsv_colors)

    return all_hsv_colors

all_rgb_colors = np.asarray(axis_bounded_cloud.colors)
all_hsv_colors = convert_rgb_2_hsv(all_rgb_colors)


In [15]:
def hsv_filter(hsv_color): #fruit
    # hsv_color is by 0 - 1 range
    
    # implement a fruit param wherein the hsv filter changes based on this
    # maybe a way where in the initial of the code, we give the hsv values of the fruits and then it can run the code??

    # # H -> 0 - 179
    # RED
    # low_h = 0/360
    # high_h = 25/360
    # # strawberry -> H S V
    # # min [  0 146 190]
    # # max [179 158 214]

    # low_h_1 = 325/360
    # high_h_1 = 360/360

    # # 330 to 30 degree

    # # S -> 0,255
    # low_s = 0 #0.3
    # high_s = 100/100

    # # V -> 0,255
    # low_v = 0
    # high_v = 1

    # yellow
    # H -> 0 - 179
    # low_h = 50/360
    # high_h = 60/360

    # low_s = 0.3
    # high_s = 100/100

    # low_v = 0.3
    # high_v = 1

    # pink flowers
    low_h1 = 290/360
    high_h1 = 360/360
    low_h2 = 0/360
    high_h2 = 20/360

    low_s = 0/255
    high_s = 255/255

    low_v = 0/255
    high_v = 255/255

    if (low_h1 <= hsv_color[0] <= high_h1) and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return True
    else:
        return False
    

In [16]:
new_points = []
new_colors = []
new_colors_RGB_u = []
for index, new_color in enumerate(all_hsv_colors):
    new_point = axis_bounded_cloud.points[index]
    new_color_RGB = all_rgb_colors[index]
    if hsv_filter(new_color):
        new_colors_RGB_u.append(new_color_RGB)
        new_points.append(new_point)
        new_colors.append(new_color)

new_colors_RGB_u = np.asarray(new_colors_RGB_u)
new_colors = np.asarray(new_colors)
new_points = np.asarray(new_points)

filtered_point_cloud = o3d.geometry.PointCloud()
filtered_point_cloud.points = o3d.utility.Vector3dVector(new_points)
filtered_point_cloud.colors = o3d.utility.Vector3dVector(new_colors_RGB_u)
visualizer(filtered_point_cloud)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [45]:
cl, ind = filtered_point_cloud.remove_statistical_outlier(nb_neighbors=70,
                                                    std_ratio=1.0)
cl.estimate_normals()
visualizer(cl)


In [46]:
# sphere = pyrsc.Sphere()
# # s = 3
# # removes the soil
# _, _,  best_inliers_plane = sphere.fit(np.asarray(cl.points), thresh=0.050, maxIteration=40)

# inlier_cloud_plane = cl.select_by_index(best_inliers_plane)
# outlier_cloud_plane = cl.select_by_index(best_inliers_plane, invert=True)

# # outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# # inlier_cloud_plane.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_plane],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [47]:
# cl = outlier_cloud_plane

In [48]:
# sphere = pyrsc.Plane()
# # s = 3
# equation, best_inliers_plane = sphere.fit(np.asarray(cl.points), thresh=0.04, maxIteration=40)

# inlier_cloud_plane = cl.select_by_index(best_inliers_plane)
# outlier_cloud_plane = cl.select_by_index(best_inliers_plane, invert=True)

# # outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# # inlier_cloud_plane.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_plane],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [49]:
filtered_point_cloud, _ = cl.remove_statistical_outlier(nb_neighbors=5, std_ratio=0.5)

visualizer(filtered_point_cloud)

In [50]:
filtered_point_cloud_r, _ = filtered_point_cloud.remove_radius_outlier(nb_points=10, radius=0.01)

visualizer(filtered_point_cloud_r)


In [51]:
# def max_red(points):
#     # Stricter filter to get just strawberry
#     low_h = 0/360
#     high_h = 25/360

#     low_h_1 = 325/360
#     high_h_1 = 360/360

#     low_s = 0.5
#     high_s = 100/100

#     low_v = 0.5
#     high_v = 1

#     H_mask = ((low_h <= points[:, 0]) & (points[:, 0] <= high_h)) | ((low_h_1 <= points[:, 0]) & (points[:, 0] <= high_h_1))
#     S_mask = (low_s <= points[:, 1]) & (points[:, 1] <= high_s)
#     V_mask = (low_v <= points[:, 2]) & (points[:, 2] <= high_v)

#     final_mask = H_mask & S_mask & V_mask

#     return np.sum(final_mask)        

In [52]:
# from sklearn.cluster import KMeans

# features = np.array(filtered_point_cloud_r.points)

# kmeans = KMeans(n_clusters=19, random_state=0).fit(features)
# red_cluster_index = np.argmin(np.mean(kmeans.cluster_centers_, axis=1))
# color = [
#     [255, 0, 0],   # Red
#     [0, 255, 0],   # Green
#     [0, 0, 255],   # Blue
#     [255, 255, 0], # Yellow
#     [255, 0, 255], # Magenta
#     [0, 255, 255], # Cyan
#     [0, 128, 0],   # Green (dark)
#     [0, 0, 128],   # Navy
#     [128, 128, 0],  # Olive
#     [128, 0, 128],  # Purple
#     [0, 128, 128],  # Teal
#     [128, 128, 128],# Gray
#     [255, 165, 0],  # Orange
#     [255, 192, 203],# Pink
#     [0, 0, 0],      # Black
#     [128, 0, 0],    # Maroon (dark)
#     [0, 128, 0],    # Green (medium)
#     [0, 0, 128],
#     [128, 128, 128]
# ]

# outlier_cloud_sph = []
# final_segments  = []

# avg = []

# for i in range(len(kmeans.cluster_centers_)):
#     temp_p = np.where(kmeans.labels_ == i)
#     temp_sphere = filtered_point_cloud_r.select_by_index(temp_p[0])
#     temp_points = np.asarray(filtered_point_cloud_r.colors)[kmeans.labels_ == i]
#     temp_avg = max_red(temp_points)
#     # temp_avg = np.average(temp_points, axis=0)
#     avg.append(temp_avg)
#     deep_copy_seg = copy.deepcopy(temp_sphere)
#     final_segments.append(deep_copy_seg)
#     outlier_cloud_sph.append(temp_sphere)
#     temp_sphere.paint_uniform_color(color[i])

# # visualizer(filtered_point_cloud)

# geometries_to_visualize = [outlier_cloud_sph[i] for i in range(len(kmeans.cluster_centers_))]  # Adjust the range as needed

# o3d.visualization.draw_geometries(geometries_to_visualize,
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])



In [53]:
# def fit_sphere(cld):

#     sphere = pyrsc.Sphere()
#     # s = 3
#     # removes the soil
#     _, _,  best_inliers_plane = sphere.fit(np.asarray(cld.points), thresh=0.090, maxIteration=40)

#     inlier_cloud_plane = cld.select_by_index(best_inliers_plane)
#     outlier_cloud_plane = cld.select_by_index(best_inliers_plane, invert=True)

#     outlier_cloud_plane.paint_uniform_color([1, 0, 0])
#     inlier_cloud_plane.paint_uniform_color([0, 1, 0])

#     # o3d.visualization.draw_geometries([outlier_cloud_plane, inlier_cloud_plane],
#     #                                     zoom=0.5,
#     #                                     front=[1, 0, 0],
#     #                                     lookat=np.mean(np.asarray(cld.points), axis=0),
#     #                                     up=[0, 0, 1])

#     return len(inlier_cloud_plane.points), len(outlier_cloud_plane.points)

In [54]:
color = [
    [255, 0, 0],   # Red
    [0, 255, 0],   # Green
    [0, 0, 255],   # Blue
    [255, 255, 0], # Yellow
    [255, 0, 255], # Magenta
    [0, 255, 255], # Cyan
    [0, 128, 0],   # Green (dark)
    [0, 0, 128],   # Navy
    [128, 128, 0],  # Olive
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 128],# Gray
    [255, 165, 0],  # Orange
    [255, 192, 203],# Pink
    [0, 0, 0],      # Black
    [128, 0, 0],    # Maroon (dark)
    [0, 128, 0],    # Green (medium)
    [0, 128, 128],
    [128, 128, 128]
]

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        filtered_point_cloud_r.cluster_dbscan(eps=0.02, min_points=40, print_progress=True))

max_label = labels.max()
outlier_cloud_sph_2 =[]
l = 0
seg_avg = []
for i in range(-1, max_label+1):
    indexes = np.argwhere(labels == i).flatten()
    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(np.asarray(filtered_point_cloud_r.points)[indexes])
    new_pc.colors = o3d.utility.Vector3dVector(np.asarray(filtered_point_cloud_r.colors)[indexes])
    if l >= 19:
        l = 0
    # new_pc.paint_uniform_color(color[l])
    l+= 1
    # new_pc.colors = o3d.utility.Vector3dVector(np.asarray(filtered_point_cloud_r.colors)[indexes])
    temp_avg = np.std(np.asarray(filtered_point_cloud_r.colors)[indexes], axis=0)
    seg_avg.append(temp_avg)
    outlier_cloud_sph_2.append(new_pc)
    # visualizer(new_pc)
    # print(l)

if show_pcl:
    o3d.visualization.draw_geometries(outlier_cloud_sph_2,
                                        zoom=0.5,
                                        front=[1, 0, 0],
                                        lookat=[0, 0, 0],
                                        up=[0, 0, 1])


[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2


c:\Users\vmuriki3\AppData\Local\miniconda3\envs\farmbot\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\vmuriki3\AppData\Local\miniconda3\envs\farmbot\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\vmuriki3\AppData\Local\miniconda3\envs\farmbot\Lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [55]:
idx = 0
max_points = 0
for i in range(len(outlier_cloud_sph_2)):
    tmp_cld = outlier_cloud_sph_2[i]
    if len(tmp_cld.points) > max_points:
        max_points = len(tmp_cld.points)
        idx = i

print(idx)

1


In [56]:
visualizer(outlier_cloud_sph_2[idx])

In [57]:
cld = outlier_cloud_sph_2[idx]

This is the point cloud with only the cube

In [58]:
cluster = outlier_cloud_sph_2[idx]
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=cluster.get_center())
# Create a visualizer object
vis = o3d.visualization.Visualizer()

# Add point cloud to visualizer/
vis.create_window()
vis.add_geometry(cluster)

# Create a coordinate frame (axes)
vis.add_geometry(mesh_frame)

# Run the visualizer
vis.run()
vis.destroy_window()# print(np.min(outlier_cloud_sph_2[idx].points, axis=0))
# print(np.max(outlier_cloud_sph_2[idx].points, axis=0))


In [59]:
# cluster = cld
# # cluster.paint_uniform_color((0, 0, 0))

# max_val = np.max(np.asarray(cluster.points), axis=0)
# min_val = np.min(cluster.points, axis=0)

# min_pt = o3d.geometry.PointCloud()
# min_pt.points = o3d.utility.Vector3dVector([min_val])
# min_pt.paint_uniform_color([0, 1, 0])

# max_pt = o3d.geometry.PointCloud()
# max_pt.points = o3d.utility.Vector3dVector([max_val])
# max_pt.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([max_pt, min_pt, cluster ])

# print(min_val)
# print(max_val)

# points_np = np.asarray(cluster.points)
# min_x_idx = np.argmin(points_np[:, 0])
# max_x_idx = np.argmax(points_np[:, 0])
# min_y_idx = np.argmin(points_np[:, 1])
# max_y_idx = np.argmax(points_np[:, 1])
# min_z_idx = np.argmin(points_np[:, 2])
# max_z_idx = np.argmax(points_np[:, 2])

In [60]:
# extremes = o3d.geometry.PointCloud()
# # , points_np[min_y_idx], points_np[max_y_idx], points_np[min_z_idx], points_np[max_z_idx]
# extremes.points = o3d.utility.Vector3dVector([points_np[min_x_idx], points_np[max_x_idx]])
# extremes.paint_uniform_color([1, 0, 0])

# o3d.visualization.draw_geometries([extremes, cld, mesh_frame, max_pt, min_pt])

In [61]:
# # Rotating the point cloud
# test_cld = cld
# points = np.asarray(test_cld.points)

# # points_centered = points - np.mean(points, axis=0)
# # cov_matrix = np.cov(points_centered, rowvar=False)

# # covariance
# mean, cov_matrix = test_cld.compute_mean_and_covariance()
# # Perform Principal Component Analysis (PCA)
# eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# # Sort eigenvectors by eigenvalues (descending order)
# sorted_indices = np.argsort(eigenvalues)[::-1]
# eigenvectors = eigenvectors[:, sorted_indices]

# # Local coordinate axes (eigenvectors)
# local_x_axis = eigenvectors[:, 0]
# local_y_axis = eigenvectors[:, 1]
# local_z_axis = eigenvectors[:, 2]

# print("Local X-axis:", np.arccos(local_x_axis))
# print("Local Y-axis:", np.arccos(local_y_axis))
# print("Local Z-axis:", np.arccos(local_z_axis))

# local_x_axis /= np.linalg.norm(local_x_axis)
# local_y_axis /= np.linalg.norm(local_y_axis)
# local_z_axis /= np.linalg.norm(local_z_axis)

# # Construct rotation matrix
# rotation_matrix = np.column_stack((local_x_axis, local_y_axis, local_z_axis))
# print(rotation_matrix)

# centroid = np.mean(points, axis=0)

# aligned_points = np.dot(points - centroid, rotation_matrix.T) + centroid

# # Create Open3D point cloud with aligned points
# aligned_cube_pcd = o3d.geometry.PointCloud()
# aligned_cube_pcd.points = o3d.utility.Vector3dVector(aligned_points)

# R = aligned_cube_pcd.get_rotation_matrix_from_xyz((np.pi, 0, np.pi/4))
# aligned_cube_pcd.rotate(R, center=aligned_cube_pcd.get_center())

# # Visualize original and aligned cubes
# o3d.visualization.draw_geometries([aligned_cube_pcd,mesh_frame])


In [62]:
# max_val = np.max(aligned_cube_pcd.points, axis=0)
# min_val = np.min(aligned_cube_pcd.points, axis=0)

# extremes = o3d.geometry.PointCloud()
# # , points_np[min_y_idx], points_np[max_y_idx], points_np[min_z_idx], points_np[max_z_idx]
# extremes.points = o3d.utility.Vector3dVector([max_val, min_val])
# extremes.paint_uniform_color([1, 0, 0])

# o3d.visualization.draw_geometries([aligned_cube_pcd,mesh_frame, extremes])

In [63]:
def scale_cube(x_min, x_max):
    actual_side_length = 0.02
    dist = x_max - x_min
    scale = actual_side_length/dist

    return scale

# scale_cube_temp = scale_cube(points_np[min_x_idx][0], points_np[max_x_idx][0])
# print(scale_cube_temp)

In [64]:
# def convert_rgb_2_hsv(all_rgb_colors):
#     all_hsv_colors_segement = []

#     for i in range(len(all_rgb_colors)):
#         temp_color = all_rgb_colors[i]
#         temp = colorsys.rgb_to_hls(temp_color[0], temp_color[1], temp_color[2]) #bt 0 to 1
#         all_hsv_colors_segement.append([temp[0] * 360, temp[1]*100, temp[2]*100])

#     all_hsv_colors_segement = np.asarray(all_hsv_colors_segement)

#     return all_hsv_colors_segement

# all_hsv_colors_segement = convert_rgb_2_hsv(seg_avg)

# print(all_hsv_colors_segement)

# index = np.argmin(all_hsv_colors_segement, axis=0)[0]

# print(index)
# # print(outlier_cloud_sph[index])
# # print(avg)
# # print(np.argmax(avg))
# visualizer(outlier_cloud_sph_2[index])


In [65]:
visualizer(cld)

In [66]:
# o3d.io.write_point_cloud("filtered_pcd_final.xyz", filtered_point_cloud)


In [67]:
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(cld,
                                                            voxel_size=0.001)

In [68]:
o3d.visualization.draw_geometries([voxel_grid])

In [95]:
cld = o3d.io.read_point_cloud("3dmodels\cube_only_extracted.ply")

# Cube 1 -> 8.13
# Cube 2 -> 11.86621893003967
# Cube 3 -> 7.35, 11.56398169309081


In [96]:
# Visualize cloud and edit
vis = o3d.visualization.VisualizerWithEditing()
vis.create_window()
vis.add_geometry(cld)
vis.run()
vis.destroy_window()
print(vis.get_picked_points())

[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #181 (0.024, 0.045, 0.23) to add in queue.
[Open3D INFO] Picked point #236 (0.018, 0.029, 0.25) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D INFO] Picked point #1487 (0.029, 0.011, 0.24) to add in queue.
[Open3D INFO] Picked point #1314 (0.012, 0.0039, 0.23) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[181, 236, 1487, 1314]


In [97]:
def euclidean_distance(color1, color2):
    return np.linalg.norm(color1 - color2)

# HSV representation of red
avg_dist = 0
for i in range(len(vis.get_picked_points())-1):
    pt = np.asarray(filtered_point_cloud.points)[vis.get_picked_points()[i]]
    next_pt = np.asarray(filtered_point_cloud.points)[vis.get_picked_points()[i+1]]
    dist = euclidean_distance(pt, next_pt)
    avg_dist += dist
    print(dist)

print("Avg:", avg_dist/(len(vis.get_picked_points())-1))


0.02014931910571491


IndexError: index 1487 is out of bounds for axis 0 with size 704

In [71]:
# import numpy as np

# # Example data (replace with your actual data)
# points = np.asarray(filtered_point_cloud.points)  # Example: XYZ coordinates
# colors = np.asarray(filtered_point_cloud.colors)  # Example: RGB colors (0-255)

# # Normalize colors to 0-1 range
# # colors_normalized = colors / 255.0

# # Concatenate points and colors
# point_cloud_data = np.concatenate((points, colors), axis=1)

# # Save to .txt file
# np.savetxt("example_point_cloud.txt", point_cloud_data, fmt='%.6f %.6f %.6f %.3f %.3f %.3f')


In [72]:
# Get the center of the strawberry
center_strawberry = cld.get_center()

center_pc = o3d.geometry.PointCloud()
center_pc.points = o3d.utility.Vector3dVector(np.asarray([center_strawberry]))
center_pc.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([cld, center_pc])

In [73]:
# Last Outlier removal if any remain
filtered_point_cloud, _ = cld.remove_statistical_outlier(nb_neighbors=10, std_ratio=0.2)
visualizer(filtered_point_cloud)

In [74]:
filtered_point_cloud_r, _ = filtered_point_cloud.remove_radius_outlier(nb_points=15, radius=0.1)

visualizer(filtered_point_cloud_r)


In [75]:
o3d.visualization.draw_geometries([filtered_point_cloud_r])

In [76]:
# from vedo import Points, Volume
# cloud = Points(np.asarray(cld.points))

In [77]:
# vol1 = Volume()

In [78]:
# visualizer(filtered_point_cloud_r)

In [79]:
# cloud.generate_mesh()

In [80]:
# # Perform 2D Delaunay triangulation to get a surface from the point cloud
# surface = cloud.smooth_mls_2d(f=0.8)
# # Points().generate_delaunay2d()
# # Compute the volume of the resulting surface mesh
# volume = surface.volume()

In [81]:
filtered_point_cloud_r.estimate_normals()
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        filtered_point_cloud_r, depth=15)
print(mesh)
o3d.visualization.draw_geometries([mesh])

[Open3D DEBUG] Input Points / Samples: 704 / 704
[Open3D DEBUG] #   Got kernel density: 0.052 (s), 231.832 (MB) / 231.832 (MB) / 530 (MB)
[Open3D DEBUG] #     Got normal field: 0.017 (s), 240.625 (MB) / 240.625 (MB) / 530 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.447380e-03 / 1.722955e+00
[Open3D DEBUG] #       Finalized tree: 0.0679998 (s), 246.586 (MB) / 246.586 (MB) / 530 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.513 (s), 237.273 (MB) / 246.586 (MB) / 530 (MB)
[Open3D DEBUG] #Set point constraints: 0.00700021 (s), 236.848 (MB) / 246.586 (MB) / 530 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 347488 / 67048 / 330081
[Open3D DEBUG] Memory Usage: 236.852 MB
[Open3D DEBUG] # Linear system solved: 0.994 (s), 239.965 (MB) / 246.586 (MB) / 530 (MB)
[Open3D DEBUG] Got average: 0.00699997 (s), 238.516 (MB) / 246.586 (MB) / 530 (MB)
[Open3D DEBUG] Iso-Value: 5.007196e-01 = 3.525066e+02 / 7.040000e+02
[Open3D DEBUG] #          Total Solve:       8.9 (s),     246.

In [82]:
unneeded = densities < np.quantile(densities, 0.5)
mesh.remove_vertices_by_mask(unneeded)
o3d.visualization.draw_geometries([mesh])

In [83]:
# def color_bounding_box(box):
#     # visualizer(mesh)
#     vertices = np.asarray(box.get_box_points())

#     # Define the edges of the oriented bounding box
#     lines = [
#         [0, 1], [1, 2], [2, 3], [3, 0],
#         [4, 5], [5, 6], [6, 7], [7, 4],
#         [0, 4], [1, 5], [2, 6], [3, 7]
#     ]

#     # Create a LineSet to represent the box
#     line_set = o3d.geometry.LineSet(
#         points=o3d.utility.Vector3dVector(vertices),
#         lines=o3d.utility.Vector2iVector(lines)
#     )

#     # Set the color of the box (e.g., red)
#     line_set.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(lines), 1)))
#     # o3d.visualization.draw_geometries([mesh, line_set],
#     #                                 zoom=0.5,
#     #                                 front=[1, 0, 0],
#     #                                 lookat=[-0.05272632, -0.02440209, 0.48188474],
#     #                                 up=[0, 0, 1])

#     return line_set

In [84]:
# # AABB box
# box_triangle = mesh.get_axis_aligned_bounding_box()
# line_set_triangle = color_bounding_box(box_triangle)
# o3d.visualization.draw_geometries([mesh, line_set_triangle, center_pc])

In [85]:
# point cloud making convex hull
hull, _ = mesh.compute_convex_hull()  #cld -> is the point cloud
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((0, 0 , 1))
# visualizer(hull_ls)
o3d.visualization.draw_geometries([hull])

In [86]:
# hull.scale(0.2517, center=hull.get_center())

In [87]:
hull.remove_degenerate_triangles()
hull.compute_vertex_normals()
hull.orient_triangles()
# o3d.visualization.draw_geometries([cld, hull])
volume = hull.get_volume() * (10 ** 6) #in cm^3

In [88]:
o3d.visualization.draw_geometries([cld, hull])

In [94]:
o3d.io.write_point_cloud("sphere_only_extracted.ply", cld)


True

In [90]:
volume

5.664602155131283

In [91]:
voxel_grid_mesh = o3d.geometry.VoxelGrid.create_from_triangle_mesh(hull,
                                                            voxel_size=0.001)

In [92]:
o3d.visualization.draw_geometries([voxel_grid_mesh])

In [93]:
len(voxel_grid_mesh.get_voxels()) * 10^9
# 39793 voxel size so mm?

24003